In [174]:
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import re
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

In [228]:
#Data preprocessing
path = './data'
prokaryotes = pd.read_csv(os.path.join(path, 'prokaryotes_348columns.csv'))
droplist = [0,1,2,3,4,5,6,8,9,10,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,35,36,37,40,41,42,45,282,283,284,285,286,287,288,289,290,291,292,293,294,300,301,302,303,304,305,306,327]
prokaryotes = prokaryotes.drop(prokaryotes.columns[droplist], axis=1)
prokaryotes.replace([np.inf, -np.inf], np.nan, inplace=True)
prokaryotes = prokaryotes.fillna(0)
prokaryotes = prokaryotes.reset_index(drop=True)

C:\Users\tobia\AppData\Local\Temp\ipykernel_3628\867054028.py:3: DtypeWarning: Columns (6,19,20,21,23,24,25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  prokaryotes = pd.read_csv(os.path.join(path, 'prokaryotes_348columns.csv'))


In [227]:
prokaryotes.columns.get_loc('HydrophobicAA')

295

In [229]:
# Splitting data into X (features) and y (target)
X = prokaryotes.drop('meltPoint', axis=1)
y = prokaryotes['meltPoint']
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
pca = PCA(n_components=25)
X_pca = pca.fit_transform(X_scaled)
X_train_pca, X_test_pca, y_train_pca, y_test_pca = train_test_split(X_pca, y, test_size=0.2, random_state=1)
gbr = GradientBoostingRegressor()
gbr.fit(X_train_pca, y_train_pca)
y_pred_pca = gbr.predict(X_test_pca)
mse_pca = mean_squared_error(y_test_pca, y_pred_pca)
r2_pca = r2_score(y_test_pca, y_pred_pca)
print(f"PCA Data - MSE: {mse_pca}, R2: {r2_pca}")

PCA Data - MSE: 63.7160764535022, R2: 0.7413429927613763


In [230]:
def fasparse(faspath):
    helixl = []
    sheetl = []
    data = np.loadtxt(faspath, dtype={'names': ('index', 'col1', 'col2', 'val1', 'val2', 'val3'),'formats': ('i4', 'S1', 'S1', 'f4', 'f4', 'f4')})
    indiceshelix = data['index'][data['col2'] == b'H']
    indicessheet = data['index'][data['col2'] == b'E']
    helix = np.split(indiceshelix, np.where(np.diff(indiceshelix) != 1)[0]+1)
    sheet = np.split(indicessheet, np.where(np.diff(indicessheet) != 1)[0]+1)
    return [helix, sheet]

In [247]:
# SPARC: "Sequence-based Protein Attribute-deRived Melt Point Calculator"
def SPARC(data:str, name, datapath, directoryS4):
    """
    data: string, protein sequence
    name: arbitrary name for the protein
    datapath: string, path to data directory, only "/" allowed
    directoryS4: string, compltete path to directory of S4pred, only "/" allowed
    """
    import numpy as np
    import pandas as pd
    import os
    from sklearn.preprocessing import StandardScaler
    from sklearn.decomposition import PCA
    from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
    from sklearn.metrics import mean_squared_error, r2_score
    features = pd.Series(np.zeros(prokaryotes.shape[1]-1))
    features.index = prokaryotes.drop(columns='meltPoint').columns
    with open(os.path.join(datapath,f'{name}.fasta'), "w") as fasta_file:
        fasta_file.write(f">{name}\n{data}\n")
    os.chdir(directoryS4)
    fastapath = os.path.join(datapath,f'{name}.fasta')
    faspath = os.path.join(datapath,f'{name}.fas')
    os.system(f'python3 run_model.py "{fastapath}" > "{faspath}"')
    os.chdir('../../')
    Protlen = len(data)
    features.iloc[0] = Protlen
    HelixSparc = fasparse(faspath)[0]
    SheetSparc = fasparse(faspath)[1]
    features.iloc[1] = len(HelixSparc)  #counts
    features.iloc[2] = len(SheetSparc)
    helixind = np.concatenate(HelixSparc)
    helixseq = np.array(list(data))[list(helixind)]
    sheetind = np.concatenate(SheetSparc)
    sheetseq = np.array(list(data))[list(sheetind)]
    features.iloc[3] = len(helixind) / Protlen #percs
    features.iloc[4] = len(sheetind) / Protlen
    features.iloc[5] = np.mean(np.array([len(arr) for arr in HelixSparc])) #avg lengths
    features.iloc[6] = np.mean(np.array([len(arr) for arr in SheetSparc]))
    features.iloc[7] = features.iloc[3] + features.iloc[4]
    aapairs = features.index[8:87]
    for aa in aapairs:
        features[aa] = (data.count(aa[0]) + data.count(aa[1])) / Protlen
    for acid in ['A', 'V', 'I', 'L', 'M', 'F', 'W','N', 'Q', 'S', 'T', 'Y','D', 'E','R', 'H', 'K', 'C', 'P', 'G' ]:
        features[f'{acid}helix'] = helixseq.tolist().count(acid) / len(helixseq)
        features[f'{acid}sheet'] = sheetseq.tolist().count(acid) / len(sheetseq)
        features[acid] = data.count(acid) / Protlen
    motifs = [motif[:2] for motif in np.array(features.iloc[107:243].index)]
    for motif in motifs:
        features[f'{motif}motif'] = data.count(motif) / Protlen
    features['EALRmotif'] = data.count('EALR') / Protlen
    features['LEALmotif'] = data.count('LEAL') / Protlen
    features['HydrophobicAA'] = features['A'] + features['V'] + features['I'] + features['L'] + features['M'] + features['F'] + features['W']
    features['ChargedAA'] = features['R'] + features['H'] + features['K'] + features['D'] + features['E']
    features['PolarAA'] = features['N'] + features['Q'] + features['S'] + features['T'] + features['Y']
    featuresdf = pd.DataFrame(features).T
    features_scaled = scaler.transform(featuresdf)
    features_pca = pca.transform(features_scaled)
    prediction = gbr.predict(features_pca)
    
    return [prediction,features]

In [214]:
features2 = pd.Series(np. zeros(prokaryotes.shape[1]-1))
features2.index = prokaryotes.drop

In [248]:
prt420 = 'MKMTSRKLSDILKQRLQHENRSFLFDREKDTLRVEDQTTKKGITLDLPPIIAKWELKKDEAIDEIVYYVSEAMTAMEGKAQEMTGKETRIYPVIRSTSFPDKSSEDIPLIYDDHTAETRIYYALDLGKTYRLIDQRMLEKENWTKERIRETAAFNLRSLPTVVKEDTVAGNYFYFFRANDGYDASRILNEAILNEYKQHAEGELAISVPHQDVLILADIRNESGYDILGQMSMSFFAGGTVPITALSFLYNEGKLEPVFILAKSRPKKD'
sparctest = 'ARVCNMPREQDERVRCYNMKLPRWQASDPOILLINCVRTQWTILLPCVFGDQVY'
O = SPARC(prt420,'sparctest',r'C:/Users/tobia/OneDrive/Documents/Uni/FS 4/Bioinfo Projekt/topic04_02/data/fastas',r'C:/Users/tobia/OneDrive/Documents/Uni/FS 4/Bioinfo Projekt/topic04_02/data/s4pred')

In [249]:
O[0]

array([46.93326437])

In [244]:
Odf = pd.DataFrame(O).T
Odf_scaled = scaler.transform(Odf)